# neo4j graph fun

# Preliminaries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from pprint import pprint
import neo4j
import tools.neotools as nj

In [ ]:
import hashlib
from hashlib import sha256

In [ ]:
from functools import lru_cache as cache
from collections import defaultdict

In [ ]:
import ipywidgets as widgets
from sidecar import Sidecar
from nnvis import NetResponsePlot

In [ ]:
from graph_utils_neo4j import NumpyStore
from nnbench import Thing

## Connecting

Need to get the `gpu-jupyter` and the `neo4j` docker containers connected. If run bare, something like:

    docker network connect gpu-jupyter_default gpu-jupyter 
    docker network connect gpu-jupyter_default neo4j
    docker network inspect gpu-jupyter_default 
    
Docker has better ways than this.

In [ ]:
driver = neo4j.GraphDatabase.driver("neo4j://172.19.0.2:7687", auth=("neo4j", "test"))

### Alive?

In [ ]:
driver.verify_connectivity()

# The Model
    Investigation -> Experiment -> multiple ResultDAGs
`ResultDAG` is

    (netState, params)-[mutation]->(netState, params)-[mutation ...
                     +-[mutation]->(netstate, params) ...
etc. `mutation` can be a learning trajectory, or an edit.

Perhaps `mutation` can be expressed in python.

Generally the results of experiments are preferred to be reproducible, but they won't always be, when they import entropy.

## Some neural nets

In [ ]:
from nn import Network, Layer, IdentityLayer, AffineLayer, MapLayer
from nnbench import NetMaker, NNMEG

In [ ]:
mnm = NetMaker(NNMEG)
xor_net = mnm('2x2tx1t')
adc_net = mnm('1x8tx8tx3t')

## ... and training data

In [ ]:
xor_training_batch = (np.array([[-0.5, -0.5],
                            [-0.5,  0.5],
                            [ 0.5,  0.5],
                            [ 0.5, -0.5]]),
                  np.array([[-0.5],
                            [ 0.5],
                            [-0.5],
                            [ 0.5]]))

In [ ]:
def adc(input):
    m = max(0, min(7, int(8*input)))
    return np.array([(m>>2)&1, (m>>1)&1, m&1]) * 2 - 1

vadc = lambda v: np.array([adc(p) for p in v])
#plot_ADC(vadc)

In [ ]:
x = np.arange(0, 1, 1.0/(8*1)).reshape(-1,1) # 1 point in each output region
adc_training_batch = (x, vadc(x))

In [ ]:
def data_from_run():
    def get_sequence(tx, rv):
        for record in tx.run("MATCH p=(head:net)-[:LEARNED*]->(tail:net) "
                             "WHERE NOT ()-[:LEARNED]->(head) "
                             "AND NOT (tail)-[:LEARNED]->() "
                             "RETURN "
                             "head, "
                             "[x IN nodes(p) | x.ksv] as ksvs, "
                             "[x IN nodes(p) | x.loss] as losses "
                            ):
            rv.head = record['head']
            rv.ksvs = record['ksvs']
            rv.losses = record['losses']

    rv = Thing
    with driver.session() as session:
        session.read_transaction(get_sequence, rv)
    return rv

In [ ]:
def parameter_names_keys_from_experiment_and_procedure_names(ex_name, proc_name):
    q = """
MATCH (e:Experiment {name: $ex_name})
-[:includes]->(proc:Procedure {name: $proc_name})
-[:incorporates]->(par:Parameters)
RETURN par.name as name, par.unikey as key
"""
    return [(r['name'], r['key']) for r in 
            nj.query_read_yield(driver, q, ex_name=ex_name, proc_name=proc_name)]

In [ ]:
parameter_names_keys_from_experiment_and_procedure_names('t2', 'Train ADCs')

In [ ]:
def data_from_run(procedure_unikey):
    q = """
MATCH (par:Parameters  {unikey: $key})-[:configures]->(head:net)
MATCH p=(head)-[:LEARNED*]->(tail:net)
WHERE NOT (tail)-[:LEARNED]->()
RETURN 
    head.shorthand as shorthand,
    [x IN nodes(p) | x.ksv] as ksvs,
    [x IN nodes(p) | x.loss] as losses
"""
    r = nj.query_read_return_list(driver, q, key=procedure_unikey)
    return dict(r[0])

In [ ]:
data_from_run('MSBTDgD7m87O3xMIV3FIXA')

In [ ]:
def data_from_experiment_and_procedure_names(ex_name, proc_name):
    q = """
MATCH (e:Experiment {name: $ex_name})
-[:includes]->(proc:Procedure {name: $proc_name})
-[:incorporates]->(par:Parameters)
-[:configures]->(head:net)
MATCH p=(head)-[:LEARNED*]->(tail:net)
WHERE NOT (tail)-[:LEARNED]->()
RETURN 
    par.name as name,
    head.shorthand as shorthand,
    [x IN nodes(p) | x.ksv] as ksvs,
    [x IN nodes(p) | x.loss] as losses
ORDER BY name
"""
    return [dict(r) for r in nj.query_read_yield(driver, q, ex_name=ex_name, proc_name=proc_name)]

In [ ]:
nps = NumpyStore(driver)

@cache
def sv_from_ksv(ksv):
    return nps[ksv]

In [ ]:
def ix_or_default(seq, ix, default):
    try:
        return seq[ix]
    except IndexError:
        return default

In [ ]:
def show_adcs_n(n):
    for nrp, thing in zip(nrps, things):
        nrp(sv_from_ksv(ix_or_default(thing.ksvs, n, thing.ksvs[-1])))
    return [ix_or_default(thing.losses, n, 0) for thing in things]

In [ ]:
things = [Thing(**d) for d in data_from_experiment_and_procedure_names('t2', 'Train ADCs')]

nets = [mnm(t.shorthand) for t in things]
nrps = [NetResponsePlot(net, height='220px', margin=30, title=net.shorthand) for net in nets]

frame_w = widgets.IntSlider(min=0,
                            max=max(len(t.ksvs) for t in things)-1,
                            step=1, value=0)

# Skip the grid
plots_box = widgets.Box(tuple(nrp.fig for nrp in nrps),
                        layout=widgets.Layout(flex_flow='row wrap',
                                             justify_content='space-around'))
        
with Sidecar(title='grid') as gside:
    display(plots_box)
    
widgets.interact(show_adcs_n, n=frame_w)

___

# Scrapyard

In [ ]:
assert False, "Scrapyard below"

# Try Queries

In [ ]:
def get_sequence(tx):
    for record in tx.run("MATCH p=(head:net)-[:LEARNED*]->(tail:net)"
                         "WHERE NOT ()-[:LEARNED]->(head)"
                         "AND NOT (tail)-[:LEARNED]->()"
                         "RETURN p, [x IN nodes(p) | x.ksv] as ksvs"):
        print(record['ksvs'])
        p = record['p']
        print(type(p))
        print(record['ksvs'])
        for r in p.relationships:
            print(r.start_node['ksv'], r['ts'], r.end_node['ksv'])

with driver.session() as session:
    session.read_transaction(get_sequence)

In [ ]:
def get_sequence(tx):
    for record in tx.run("MATCH p=(head:net)-[:LEARNED*]->(tail:net) "
                         "WHERE NOT ()-[:LEARNED]->(head) "
                         "AND NOT (tail)-[:LEARNED]->() "
                         "RETURN "
                         "head, "
                         "[x IN nodes(p) | x.ksv] as ksvs, "
                         "[x IN nodes(p) | x.loss] as losses "
                        ):
        head = record['head']
        ksvs = record['ksvs']
        losses = record['losses']
        print(ksvs, losses)
        print(head)
        
with driver.session() as session:
    session.read_transaction(get_sequence)

In [ ]:
d = defaultdict(Thing)

In [ ]:
d['a'].foo = 3
d['b'].bar = lambda x: x*x

In [ ]:
d

In [ ]:
d['b'].bar(3)